In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from datetime import datetime
from pytz import timezone
datetime.now(timezone('Asia/Tokyo')).strftime('%Y/%m/%d %H:%M:%S')

def refer_args(x):
    if type(x) == 'method':
        print(*x.__code__.co_varnames.split(), sep='\n')
    else:
        print(*[x for x in dir(x) if not x.startswith('__')], sep='\n')

In [ ]:
from collections import defaultdict
from collections import Counter
import re
import string

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import numpy as np
from tqdm import tqdm

from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import gensim
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from keras.optimizers import Adam

In [ ]:
df_train = pd.read_csv('../input/nlp-getting-started/train.csv')
df_train_meta = pd.read_csv('../input/nlpwithdisastertweets-searching-real-tweets/train_tweet.csv')
df_test = pd.read_csv('../input/nlp-getting-started/test.csv')
df_test_meta = pd.read_csv('../input/nlpwithdisastertweets-searching-real-tweets/test_tweet.csv')
df_train.head(3)
df_train_meta.head(3)

In [ ]:
df_train.head(3)
df_train_meta.head(3)

In [ ]:
df_train_meta['permalink'][1]

In [ ]:
from datetime import datetime

## ハッシュタグの数を抽出
df_train_meta['nb_hashtag'] = \
        df_train_meta['hashtags'].fillna('').apply(lambda x:x.count('#'))

## リプライの数を抽出
df_train_meta['nb_mention'] = \
        df_train_meta['mentions'].fillna('').apply(lambda x:x.count('@'))

## コンペ以降のツイートデータを無視
compete_start_date = datetime(2019, 12, 20, tzinfo=timezone('UTC'))
sr_date = pd.to_datetime(df_train_meta['date'].fillna(datetime.now()))
df_train_meta[sr_date >= compete_start_date] = pd.NA

In [ ]:
df_train['exist tweet'] = df_train_meta['id'].notna()
df_train['favorites'] = df_train_meta['favorites']
df_train['retweets'] = df_train_meta['retweets']
df_train['mentions'] = df_train_meta['nb_mention']
df_train['hashtags'] = df_train_meta['hashtags']
df_train['date'] = df_train_meta['date']

In [ ]:
df_train.head(3)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
_ = sns.barplot(x='exist tweet', y='target', data=df_train, ax=ax1)
_ = sns.countplot(x='exist tweet', hue='target', data=df_train, ax=ax2)

In [ ]:
bins = (-2, -0.5, 0.5, 2.5, 9.5, 1000)
group_names = ('Unknown', '0', '1-2', '3-9', '10-')
df_train['FavGroup'] = pd.cut(df_train['favorites'].fillna(-1), bins, labels=group_names)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
_ = sns.barplot(x='FavGroup', y='target', data=df_train, ax=ax1)
_ = sns.countplot(x='FavGroup', hue='target', data=df_train, ax=ax2)

In [ ]:
bins = (-2, -0.5, 0.5, 2.5, 9.5, 1000)
group_names = ('Unknown', '0', '1-2', '3-9', '10-')
df_train['RetGroup'] = pd.cut(df_train['retweets'].fillna(-1), bins, labels=group_names)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
_ = sns.barplot(x='RetGroup', y='target', data=df_train, ax=ax1)
_ = sns.countplot(x='RetGroup', hue='target', data=df_train, ax=ax2)

In [ ]:
bins = (-2, -0.5, 0.5, 2.5, 4.5, 1000)
group_names = ('Unknown', '0', '1-2', '3-4', '5-')
df_train['MenGroup'] = pd.cut(df_train['mentions'].fillna(-1), bins, labels=group_names)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
_ = sns.barplot(x='MenGroup', y='target', data=df_train, ax=ax1)
_ = sns.countplot(x='MenGroup', hue='target', data=df_train, ax=ax2)

In [ ]:
bins = (-2, -0.5, 0.5, 1000)
group_names = ('Unknown', '0', '1-')
df_train['HashGroup'] = pd.cut(df_train['mentions'].fillna(-1), bins, labels=group_names)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
_ = sns.barplot(x='HashGroup', y='target', data=df_train, ax=ax1)
_ = sns.countplot(x='HashGroup', hue='target', data=df_train, ax=ax2)